# RAG using HuggingFace

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
loader = PyPDFDirectoryLoader('../data/')
docs = loader.load()
docs

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunked_docs = text_splitter.split_documents(docs)
chunked_docs

In [ ]:
# Embedding

hugging_face_embedding = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-small-en-v1.5",  #sentence-transformer/all-MiniLM-16-v2
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

In [ ]:
import numpy as np

np.array(hugging_face_embedding.embed_query(chunked_docs[0].page_content)).shape

In [ ]:
vector_db = FAISS.from_documents(chunked_docs[:100], hugging_face_embedding)
vector_db

In [ ]:
#  Query similarity search

query = "what is health insurance coverage?"

relevant_docs = vector_db.similarity_search(query)
print(relevant_docs[0].page_content)

In [ ]:
retriever = vector_db.as_retriever(search_type='similarity', search_kwargs = {'k':3})
print(retriever)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN','')

In [ ]:
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs = {"temperature":0.1, "max_length":500}
)

query = "what is health insurance coverage?"

hf.invoke(query)

In [ ]:
# Running huggingface model on locally using pipeline

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [ ]:
prompt_template = """
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context.

{context}

Questions: {question}

Helpful Answers:
"""

In [ ]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

In [ ]:
retrieverQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query = """
Differences in the uninsured rate by state in 2022
"""

In [ ]:
# call the QA with chain

result = retrieverQA.invoke({"query":query})
print(result['result'])